In [3]:
import preprocessing.preprocess as preprocess

In [4]:
fake_news = preprocess.parse_dataset("Fake_test.csv", "FAKE")
real_news = preprocess.parse_dataset("True_test.csv", "REAL")

fake_news_all_tokens, fake_news_tokens_per_article = preprocess.tokenize(fake_news, "fake_news")
real_news_all_tokens, real_news_tokens_per_article = preprocess.tokenize(real_news, "real_news")

# join tokens and data together
all_tokens = fake_news_all_tokens + real_news_all_tokens
tokens_per_article = fake_news_tokens_per_article + real_news_tokens_per_article

all_news = pd.concat([fake_news, real_news], axis=0)

print("Preview of parsed data:")
print(all_news)

# join the data and pass it to split data
X_train, X_test, y_train, y_test = preprocess.split_and_preprocess(all_tokens,tokens_per_article, all_news)

Parsing news dataset from file: Fake_test.csv
Setting label for news dataset: FAKE
Parsing news dataset from file: True_test.csv
Setting label for news dataset: REAL
Tokenizing fake_news dataset, this may take a few minutes...
Tokenizing real_news dataset, this may take a few minutes...
Preview of parsed data:
                                               title  \
0  FLASHBACK: KING OBAMA COMMUTES SENTENCES OF 22...   
1  APPLE’S CEO SAYS RELIGIOUS FREEDOM LAWS ARE ‘D...   
2  WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...   
3  OH NO! GUESS WHO FUNDED THE SHRINE TO TED KENNEDY   
4  BENGHAZI PANEL CALLS HILLARY TO TESTIFY UNDER ...   
5  HILLARY RODHAM NIXON: A CANDIDATE WITH MORE BA...   
6  WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...   
7  HILLARY RODHAM NIXON: A CANDIDATE WITH MORE BA...   
8  MUSLIM WOMAN ARRESTED FOR SPITTING ON HER FELL...   
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. R